In [1]:
# iris 데이터셋 로딩
from sklearn.datasets import load_iris

x, y = load_iris(return_X_y=True)
print(x[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [2]:
from sklearn.model_selection import train_test_split
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, random_state = 10, test_size = 0.2, stratify = y, shuffle = True)
len(x_tr)

120

In [3]:
from torch import nn
import torch.nn.functional as F

# 신경망모형
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        # input layer, Linear 1차함수
        # input nodes, output nodes 50
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 20)
        self.layer3 = nn.Linear(20, 3)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=0)
        return x

In [4]:
import torch
model = Model(x_tr.shape[1])  # 초기화함수의 input_dim으로 변수개수 전달
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # 최적화함수 정의
loss_fn = nn.CrossEntropyLoss()   # 손실함수
epochs = 100

In [5]:
from torch.autograd import Variable
x_train = Variable(torch.from_numpy(x_tr)).float()
y_train = Variable(torch.from_numpy(y_tr)).long()

for epoch in range(1, epochs+1):
    print("Epoch", epoch)
    # 예측값
    y_pred = model(x_train)

    # 손실함수에 예측값과 실제값 입력
    loss = loss_fn(y_pred, y_train)
    print('loss:', loss.item())

    # 경사초기화
    optimizer.zero_grad()
    loss.backward()  # 역전파
    optimizer.step()  # 가중치업데이트

Epoch 1
loss: 1.0984668731689453
Epoch 2
loss: 1.0974229574203491
Epoch 3
loss: 1.0964428186416626
Epoch 4
loss: 1.095467448234558
Epoch 5
loss: 1.0942389965057373
Epoch 6
loss: 1.0930181741714478
Epoch 7
loss: 1.0919082164764404
Epoch 8
loss: 1.0908933877944946
Epoch 9
loss: 1.0899903774261475
Epoch 10
loss: 1.0892446041107178
Epoch 11
loss: 1.0887044668197632
Epoch 12
loss: 1.088281512260437
Epoch 13
loss: 1.0879327058792114
Epoch 14
loss: 1.0876280069351196
Epoch 15
loss: 1.0873583555221558
Epoch 16
loss: 1.0871249437332153
Epoch 17
loss: 1.0869371891021729
Epoch 18
loss: 1.0868198871612549
Epoch 19
loss: 1.0867855548858643
Epoch 20
loss: 1.0867151021957397
Epoch 21
loss: 1.086606502532959
Epoch 22
loss: 1.086495280265808
Epoch 23
loss: 1.086396336555481
Epoch 24
loss: 1.0863277912139893
Epoch 25
loss: 1.0862568616867065
Epoch 26
loss: 1.086134672164917
Epoch 27
loss: 1.0859878063201904
Epoch 28
loss: 1.0858433246612549
Epoch 29
loss: 1.0857020616531372
Epoch 30
loss: 1.085557699203

In [6]:
# prediction
x_test = Variable(torch.from_numpy(x_ts)).float()
pred = model(x_test)
pred[:5]

tensor([[1.7911e-08, 8.5237e-03, 7.6710e-05],
        [2.2809e-02, 1.5496e-06, 1.3424e-09],
        [8.8213e-09, 1.5601e-03, 1.8399e-04],
        [1.9989e-09, 8.5623e-04, 8.2037e-04],
        [1.0429e-06, 3.1595e-01, 1.0237e-06]], grad_fn=<SliceBackward0>)

In [7]:
import numpy as np
np.argmax(pred.data.numpy(), axis = 1)

array([1, 0, 1, 1, 1, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       2, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_ts, np.argmax(pred.data.numpy(), axis=1))

0.9666666666666667

In [9]:
torch.save(model, 'Data/iris-torch.h5')

In [10]:
model2 = torch.load('Data/iris-torch.h5')
# np.argmax(model2(x_test[0]).data.numpy())
np.argmax(model2(x_test[0]).detach().numpy())

2

In [11]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
Model                                    --
├─Linear: 1-1                            250
├─Linear: 1-2                            1,020
├─Linear: 1-3                            63
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0

In [12]:
# input_size=(batch size, input nodes)
summary(model, input_size=(32, 4))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [32, 3]                   --
├─Linear: 1-1                            [32, 50]                  250
├─Linear: 1-2                            [32, 20]                  1,020
├─Linear: 1-3                            [32, 3]                   63
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
Total mult-adds (M): 0.04
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.02